In [5]:
import pandas as pd

In [6]:

# dataf = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/src/evaluation/figure_notebooks/figure_6/final_figures_new/smc.caqtls.tsv", sep="\t", header=0)
# dataf
# sum(dataf["IsUsed"])
# dataf.columns

In [7]:
orig = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/smc_caqtls/SMC_rasqual_results_FDR_0.05_083121.txt",  sep="\t", header=0)
preds = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/smc_caqtls/ATAC_sig_only_new/summary.mean.variant_scores_new.tsv", sep="\t", header=0)
import numpy as np
preds["jsd.sign.mean"] = np.sign(preds["logfc.mean"])* preds["jsd.mean"]
preds = preds[["chr", "pos", "allele1", "allele2", "logfc.mean", "jsd.sign.mean"]]
orig = orig.merge(preds, left_on=['Chromosome', 'SNP_position', 'Ref_allele','Alt_allele'], right_on=["chr", "pos", "allele1", "allele2"]).drop_duplicates()
orig["pos0"] = orig["SNP_position"] - 1
selected_beds = orig[['Chromosome', "pos0", 'SNP_position', 'Ref_allele', 'Alt_allele', 'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean']]


fold_0=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_0/peaks.testset.bed.gz", sep="\t", header=None)
fold_1=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_1/peaks.testset.bed.gz", sep="\t", header=None)
fold_2=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_2/peaks.testset.bed.gz", sep="\t", header=None)
fold_3=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_3/peaks.testset.bed.gz", sep="\t", header=None)
fold_4=pd.read_csv("/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_4/peaks.testset.bed.gz", sep="\t", header=None)
peaks_bed = pd.concat([fold_0, fold_1, fold_2, fold_3, fold_4])
peaks_bed[1] = peaks_bed[1] + peaks_bed[9] - 100
peaks_bed[2] = peaks_bed[1] + 200


import pybedtools
beds_var = pybedtools.BedTool.from_dataframe(selected_beds)
peak_bed =  pybedtools.BedTool.from_dataframe(peaks_bed)
sig_vars_in_bed = beds_var.intersect(peak_bed, c=True)

sig_vars_in_bed = sig_vars_in_bed.to_dataframe(names=['Chromosome', "pos0", 'SNP_position', 'Ref_allele', 'Alt_allele', 'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean', "hits"]).drop_duplicates()


sig_vars_in_bed["IsUsed"] = sig_vars_in_bed["hits"] > 0

OSError: [Errno 127] Key has expired: '/oak/stanford/groups/akundaje/anusri/chrombpnet_zenodo_uploads_reformatted/SMC_scATAC_ChromBPNet/data/fold_0/peaks.testset.bed.gz'

In [8]:
sig_vars_in_bed

NameError: name 'sig_vars_in_bed' is not defined

In [9]:
sum(sig_vars_in_bed["IsUsed"])

NameError: name 'sig_vars_in_bed' is not defined

In [6]:
sig_vars_in_bed.columns

Index(['Chromosome', 'pos0', 'SNP_position', 'Ref_allele', 'Alt_allele',
       'Log10_BH_Q', 'Effect_size', 'logfc.mean', 'jsd.sign.mean', 'hits',
       'IsUsed'],
      dtype='object')

In [7]:
orig.columns

Index(['Feature_ID', 'rsID', 'Chromosome', 'SNP_position', 'Ref_allele',
       'Alt_allele', 'Freq', 'HWE_Chisquare', 'Imp_quality', 'Log10_BH_Q',
       'Chisquare', 'Effect_size', 'Delta', 'Phi', 'Overdispersion',
       'SNP_id_region', 'Num_feature_SNPs', 'Num_tested_SNPs',
       'Num_iterations_null', 'Num_iterations_alt', 'Random_ties',
       'Log_likelihood_null', 'Convergence_status', 'Sq_corr_fSNPs',
       'Sq_corr_rSNP', 'P', 'q', 'keep', 'PeakID', 'Chr', 'Start', 'End',
       'Strand', 'Peak Score', 'Focus Ratio/Region Size', 'Annotation',
       'Detailed Annotation', 'Distance to TSS', 'Nearest PromoterID',
       'Entrez ID', 'Nearest Unigene', 'Nearest Refseq', 'Nearest Ensembl',
       'Gene Name', 'Gene Alias', 'Gene Description', 'Gene Type', 'chr',
       'pos', 'allele1', 'allele2', 'logfc.mean', 'jsd.sign.mean', 'pos0'],
      dtype='object')

In [8]:
sig_vars_in_bed.shape

(1984, 11)

In [9]:
orig.shape

(1984, 54)

In [10]:
rename_dict = {'Chromosome': "var.chr",
                'SNP_position': "var.pos_hg38",
                'Ref_allele': "var.ref",
                'Alt_allele': "var.alt",
                'Log10_BH_Q': "obs.Log10_BH_Q", 
                'Effect_size': "obs.Effect_size",
                'logfc.mean': "pred.chrombpnet.smc.varscore.logfc",
                'jsd.sign.mean': "pred.chrombpnet.smc.varscore.jsd",
                'IsUsed': "var.isused"}

In [11]:
data = sig_vars_in_bed.rename(columns=rename_dict)

In [12]:
data.columns

Index(['var.chr', 'pos0', 'var.pos_hg38', 'var.ref', 'var.alt',
       'obs.Log10_BH_Q', 'obs.Effect_size',
       'pred.chrombpnet.smc.varscore.logfc',
       'pred.chrombpnet.smc.varscore.jsd', 'hits', 'var.isused'],
      dtype='object')

In [13]:
use_cols=['var.chr', 'var.pos_hg38', 'var.ref', 'var.alt',
       'obs.Log10_BH_Q', 'obs.Effect_size',
       'pred.chrombpnet.smc.varscore.logfc',
       'pred.chrombpnet.smc.varscore.jsd', 'var.isused']

In [14]:
dataf = data[use_cols]


In [15]:
dataf_final = dataf[dataf["var.isused"]]
dataf_final = dataf_final.drop(["var.isused"], axis=1)
dataf.to_csv("full_variants/caqtls.smc.benchmarking.all.tsv", sep='\t', header=True, index=False)
dataf_final.to_csv("filtered_variants/caqtls.smc.benchmarking.chrombpnet.tsv", sep='\t', header=True, index=False)

In [16]:
dataf_final.shape

(386, 9)

In [4]:
dataf.shape

NameError: name 'dataf' is not defined

In [3]:
dataf.columns

NameError: name 'dataf' is not defined

In [11]:
readme_data = {'var.chr' : "Chromosome of the variant (GRCh38)", 
        'var.pos_hg38': "Position of the variant (GRCh38, 1-based)",
        'var.ref': "Reference Allele for the variant", 
        'var.alt': "Alternate Allele for the variant",        
       'obs.Effect_size': "Observed effect size", 
        'obs.Log10_BH_Q': "Significance value of the Observed effect size",
       'var.isused': "True if variant is used in final ChromBPNet benchmarking",
       'pred.chrombpnet.smc.varscore.logfc': "ChromBPNet logFC predictions in SMC",
       'pred.chrombpnet.smc.varscore.jsd':  "ChromBPNet JSD predictions in SMC", 
      
 }


In [12]:
f = open("full_variants/caqtls.smc.benchmarking.all.README", "w")
f.write("# TSV Field Header Descriptions \n \n")
for val in readme_data:
    print(val)
    f.write(val+" : "+readme_data[val]+"\n")
f.write("\n")
f.write("# Experiment-Specific Headers and IDs  \n")
f.close()

var.chr
var.pos_hg38
var.ref
var.alt
obs.Effect_size
obs.Log10_BH_Q
var.isused
pred.chrombpnet.smc.varscore.logfc
pred.chrombpnet.smc.varscore.jsd
